# BoxBunny Unit Tests
Essential unit tests for validating system components.

## Setup Environment
Run this first to configure the shell environment.

In [ ]:
%%bash
cat > /tmp/boxbunny_env.sh <<'SH'
source ~/miniconda3/etc/profile.d/conda.sh
conda activate boxing_ai
export PYTHONNOUSERSITE=1
export LD_LIBRARY_PATH=$CONDA_PREFIX/lib:$LD_LIBRARY_PATH
export SETUPTOOLS_USE_DISTUTILS=stdlib
source /opt/ros/humble/setup.bash
source /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/install/setup.bash 2>/dev/null || true
SH
echo "Environment ready"

## Build Workspace
**Critical:** Run this to build `boxbunny_msgs` and other packages.

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws
colcon build --symlink-install
source install/setup.bash

## 1. GUI Design Test
Test the main robot GUI appearance and layout.
**Expectation:** The main BoxBunny GUI opens. You can navigate menus to verify design.

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/src/boxbunny_gui/boxbunny_gui

# Run GUI directly with conda python (has PySide6)
python gui_main.py

## 2. IMU Calibration Test
Test the IMU sensors and punch calibration interface.
**Expectation:** The IMU Calibration GUI opens. You can see accelerometer/gyroscope data.

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws

# Rebuild IMU package for config changes
colcon build --packages-select boxbunny_imu --symlink-install > /dev/null 2>&1
source install/setup.bash

# Start IMU nodes in background
ros2 run boxbunny_imu mpu6050_node &
ros2 run boxbunny_imu imu_punch_classifier &
sleep 1

# Run GUI directly with conda python (has PySide6)
cd src/boxbunny_imu/boxbunny_imu
python imu_punch_gui.py

## 3. Reaction Drill Test
Test the **Reaction Time Drill** using the Pose/Action model.
**Expectation:** 
1. **Everything starts automatically** (Drill Manager, AI, GUI).
2. GUI defaults to **Action / Pose Model** mode.
3. You should see video + skeleton overlay immediately.
4. Click **START REACTION DRILL** - each session runs **3 trials**.
5. **Flow per trial:** Wait... → PUNCH! → (you punch) → ✓ → Wait... → repeat
6. If you punch before "PUNCH!" appears: shows **"EARLY!"** → returns to Wait... → shows PUNCH! again.
7. After 3 trials, drill completes and shows your average reaction time.

In [ ]:
%%bash
source /tmp/boxbunny_env.sh

# AGGRESSIVELY kill conflicting camera nodes
pkill -f realsense2_camera_node || true
pkill -f live_infer_rgbd.py || true
# Kill any ros launch processes that might restart nodes
pkill -f "ros2 launch boxbunny_bringup vision_only.launch.py" || true
sleep 2

cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws

# Rebuild drills package to install new launch file
colcon build --packages-select boxbunny_drills --symlink-install > /dev/null
source install/setup.bash

# Launch Unified Reaction Drill
ros2 launch boxbunny_drills reaction_drill.launch.py

## 4. Punch Detection Test (Color Tracking)
Test punch detection using color tracking (standalone).
**Glove Colors:**
- LEFT glove (RED) = JAB
- RIGHT glove (GREEN) = CROSS

**Expectation:** 
1. A dedicated Punch Detection GUI opens with live camera feed.
2. Bounding boxes appear around detected gloves (red/green).
3. When you throw a punch (glove approaches camera quickly), it detects JAB or CROSS.
4. Punch counter and log show all detected punches.

In [ ]:
%%bash
source /tmp/boxbunny_env.sh

# Kill any existing camera processes
pkill -f realsense2_camera_node || true
pkill -f live_infer_rgbd.py || true
pkill -f punch_detection_gui.py || true
sleep 1

cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws

# Rebuild drills package
colcon build --packages-select boxbunny_drills --symlink-install > /dev/null
source install/setup.bash

# Launch Punch Detection (Color Tracking)
ros2 launch boxbunny_drills punch_detection.launch.py

## 5. Height Calibration Test (Standalone)
Calibrate and verify user height using Pose + Depth data.
**This test is completely standalone** - runs its own camera and pose model.

**Expectation:**
1. A dedicated Height Calibration GUI opens with live camera feed.
2. Skeleton overlay and bounding box appear on your body.
3. Position yourself 2-3 meters from camera with FULL BODY visible.
4. Click CALIBRATE HEIGHT and hold still during the countdown.
5. System averages height samples over 2 seconds and displays calibrated height.

In [ ]:
%%bash
source /tmp/boxbunny_env.sh

# Kill any existing camera processes
pkill -f realsense2_camera_node || true
pkill -f live_infer_rgbd.py || true
pkill -f height_calibration_gui.py || true
sleep 1

cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws

# Rebuild drills package
colcon build --packages-select boxbunny_drills --symlink-install > /dev/null
source install/setup.bash

# Launch Height Calibration
ros2 launch boxbunny_drills height_calibration.launch.py

[0.646s] WARNING:colcon.colcon_core.package_selection:Some selected packages are already built in one or more underlay workspaces:
	'boxbunny_drills' is in: /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/install/boxbunny_drills
If a package in a merged underlay workspace is overridden and it installs headers, then all packages in the overlay must sort their include directories by workspace order. Failure to do so may result in build failures or undefined behavior at run time.
If the overridden package is used by another package in any underlay, then the overriding package in the overlay must be API and ABI compatible or undefined behavior at run time may occur.

If you understand the risks and want to override a package anyways, add the following to the command line:
	--allow-overriding boxbunny_drills

This may be promoted to an error in a future release of colcon-override-check.


[INFO] [launch]: All log files can be found below /home/boxbunny/.ros/log/2026-01-29-22-32-54-225307-ubuntu-84364
[INFO] [launch]: Default logging verbosity is set to INFO
[INFO] [python3-1]: process started with pid [84365]
